# Embeddings

The aim of this notebook is to generate embeddings for the two main approaches that we use for generating graphs:

1. **embedding approach**: simply use [e5-large](https://huggingface.co/intfloat/e5-large) model for generating feature vectors from plots;
2. **genre approach**: first let the [bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli) predict the probabilities of belonging to a genre and a theme, then merge the vectors into an embedding.

## Imports

In [2]:
import json
import os

import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from transformers import pipeline
import torch

from sentence_transformers import SentenceTransformer

import os
import sys
sys.path.append(os.path.abspath('..'))

from scripts.helpers import get_embedding, get_classification_from_json

In [3]:
root_path = os.path.dirname(os.path.abspath(""))

## Data

Note that data preprocessing has been done in [preprocess.ipynb](preprocess.ipynb). The notebook will clean, merge and save the data to `data/processed`.

In [4]:
df = pd.read_csv('../data/processed/preprocessed.csv', index_col=0)
print(df.shape)
df.head(5)

(22394, 10)


,name,release_year,rating,num_votes,wikipedia_id,plot,freebase_id,languages,countries,genres
0,Miss Jerry,1894.0,5.3,207,10109752,After finding out that her father is suffering...,/m/02q23xk,['Silent film'],['United States of America'],"['Short Film', 'Silent film', 'Indie', 'Black-..."
1,The Corbett-Fitzsimmons Fight,1897.0,5.3,484,28703057,The film no longer exists in its entirety; how...,/m/0czdh_n,[],[],['Sports']
2,The Story of the Kelly Gang,1906.0,6.0,855,142995,The Story of the Kelly Gangs tone is of sorrow...,/m/0120y4,['English Language'],['Australia'],"['Crime Fiction', 'Silent film', 'Biography', ..."
3,Robbery Under Arms,1907.0,4.3,25,32986669,Key scenes of the film included the branding o...,/m/04p7yxx,['Silent film'],['Australia'],"['Silent film', 'Drama']"
4,"Captain Midnight, the Bush King",1911.0,5.4,18,32987200,Edgar Dalimore is the son of wealthy station o...,/m/0h569x9,['Silent film'],[],['Silent film']


## Embedding Approach

We're using the model described in this [paper](https://arxiv.org/pdf/2212.03533.pdf), which can be downloaded [here](https://huggingface.co/intfloat/e5-large-v2). To download the model, run in the `models` directory:

```
git lfs install
git clone https://huggingface.co/intfloat/e5-large-v2
```

In [5]:
model = SentenceTransformer('../models/e5-large-v2')
decades = np.arange(1910, 2020, 10)
input_template = 'query: {}'

In [6]:
for decade in decades:
    decade_df = df[(df.release_year >= decade) & (df.release_year < decade + 10)]
    
    # create dictionary with embeddings for all the movies
    embeddings = {}
    
    for index, row in decade_df.iterrows():
        plot = input_template.format(row['plot'])
        movie = row['name']
        embeddings[movie] = get_embedding(plot, model)
        
    with open(os.path.join(root_path, 'data', 'embeddings', 'embedding', f'plots{decade}s.json'), "w") as outfile: 
        json.dump(embeddings, outfile)

## Genre Approach

We first let the model predict the probabilities and then convert them into embeddings.

In [ ]:
genre_labels = ['Drama', 'Comedy', 'Action', 'Horror', 'Romance', 'Thriller', 'Adventure', 'Documentary']
theme_labels = ['Science-Fiction', 'Crime', 'Fantasy', 'War', 'Mystery', 'History', 'Western', 'Family']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

In [ ]:
hypothesis_template = 'The genre of this movie is {}.'

for decade in decades:
    decade_df = df[(df.release_year >= decade) & (df.release_year < decade + 10)]

    file_path = os.path.join(root_path, 'data', 'classification', 'probabilities', f'probabilities_{decade}s.json')

    for i, row in tqdm(enumerate(decade_df.iterrows()), total=decade_df.shape[0]):

        wikipedia_id = str(row[1].wikipedia_id)
        probabilities[wikipedia_id] = {}

        # Computing probs for genres only
        genre_probs = classifier(row[1]['plot'], genre_labels, multilabel=True, hypothesis_template=hypothesis_template)
        genre_probs.pop('sequence')
        probabilities[wikipedia_id]['genres'] = genre_probs

        # Computing probs for themes only
        theme_probs = classifier(row[1]['plot'], theme_labels, multilabel=True, hypothesis_template=hypothesis_template)
        theme_probs.pop('sequence')
        probabilities[wikipedia_id]['themes'] = theme_probs

        # Computing probs for genres and themes
        full_probs = classifier(row[1]['plot'], genre_labels + theme_labels, multilabel=True, hypothesis_template=hypothesis_template)
        full_probs.pop('sequence')
        probabilities[wikipedia_id]['full'] = full_probs

    with open(file_path, "w") as outfile: 
        json.dump(probabilities, outfile)

In [ ]:
decades = np.arange(1910, 2020, 10)

for decade in decades:

    probabilities = get_classification_from_json(decade)    
    embeddings = {}
    
    for movie, probability in probabilities.items():
       
        labels = probability['genres']['labels'] + probability['themes']['labels']
        scores = probability['genres']['scores'] + probability['themes']['scores']
    
        # sort the labels and reorder the scores accordingly
        _, scores = zip(*sorted(zip(labels, scores)))
    
        # normalize the embedding
        embedding = np.array(scores)
        norm_embedding = embedding / np.linalg.norm(embedding)
        
        embeddings[str(movie)] = list(norm_embedding)
        
    with open(os.path.join(root_path, 'data', 'embeddings', 'custom_genres', f'plots_{decade}s.json'), "w") as outfile: 
        json.dump(embeddings, outfile)